# import

In [ ]:
import random
from pyensembl import EnsemblRelease
#run the next line if you have not used EnsemblRelease
#pyensembl install --release 110 --species homo_sapiens

# Related data acquire

Hct116.txt acquisition：

wget http://sg-nex-data.s3.amazonaws.com/data/annotations/m6ACE_seq_reference_table/Hct116_m6ACEsites.txt -O Hct116.txt

human.hg38.m6A.result.col29.bed acquisition:

download hg38.m6A.tar.gz in https://rna.sysu.edu.cn/rmbase3/download.php
![My Image](instru.png)
tar -zxvf hg38.m6A.tar.gz

# From .bed to ENST

In [2]:
def get_sites_for_cell_line(Ensembl_data,bed_file,search_cell_line):
    sites_write_ENST=open(search_cell_line+'_ENST.txt','w')

    f=open(bed_file+'.bed','r')
    for line in f.readlines():
        items=line.strip().split('\t')
        line_cell_lines=items[11].split(',')
        if search_cell_line not in line_cell_lines:
            continue
        g_pos=int(items[2])
        k5=items[18][18:23]
        line_ENSTs=items[14].split(',')
        for ENST in line_ENSTs:
            try:
                exons=Ensembl_data.exon_ids_of_transcript_id(ENST[:15])
            except:
                continue
            c_pos=0
            for exon in exons:
                try:
                    exon_info=Ensembl_data.exon_by_id(exon)
                except:
                    break

                exon_start,exon_end=exon_info.start,exon_info.end
                if g_pos>exon_end:
                    c_pos+=exon_end-exon_start+1
                    continue
                if g_pos>=exon_start:
                    sites_write_ENST.write('%s_%d_%s\n'%(ENST[:15],c_pos+g_pos-exon_start,k5))                
                break
    sites_write_ENST.close()
    f.close()

In [3]:
Ensembl_data=EnsemblRelease(110)

In [20]:
get_sites_for_cell_line(Ensembl_data,'human.hg38.m6A.result.col29','HepG2')

In [21]:
get_sites_for_cell_line(Ensembl_data,'human.hg38.m6A.result.col29','A549')

# From ENSG to ENST

In [22]:
def ENSG_reindex_to_ENST(Ensembl_data,sites_file):
    sites_write_ENST=open(sites_file+'_ENST.txt','w')
    f=open(sites_file+'.txt','r')
    
    c_gene=None
    c_trans={}
    while 1:
        line=f.readline()
        if not line:
            break
        gene,g_pos,k5=line.strip().split('_')
        g_pos=int(g_pos)
        
        try:
            Ensembl_data.transcript_ids_of_gene_id(gene)
        except:
            continue
        
        if gene!=c_gene:
            c_trans={}
            for t_trans in Ensembl_data.transcript_ids_of_gene_id(gene):
                c_trans[t_trans]=[]
                for t_exon in Ensembl_data.exon_ids_of_transcript_id(t_trans):
                    t_exon_info=Ensembl_data.exon_by_id(t_exon)
                    c_trans[t_trans].append([t_exon_info.start,t_exon_info.end])
            c_gene=gene

        for ENST in c_trans:
            c_pos=0
            for start_end in c_trans[ENST]:
                if g_pos>start_end[1]:
                    c_pos+=start_end[1]-start_end[0]+1
                    continue
                if g_pos>=start_end[0]:
                    sites_write_ENST.write('%s_%d_%s\n'%(ENST,c_pos+g_pos-start_end[0],k5))                
                break
    sites_write_ENST.close()
    f.close()

In [23]:
sites_file='./Hct116'
Ensembl_data=EnsemblRelease(110)
ENSG_reindex_to_ENST(Ensembl_data,sites_file)

# One of the ways to select negative sites

In [ ]:
#only a half of the negative sites is selected in this way,
#by selecting sites that is modifiable in other cell lines but not in the cell line where positive labels come from

In [3]:
def get_negative_sites(to_select_sites_files,exclude_sites_file,write_file):
    e_file=open(exclude_sites_file+'.txt','r')
    e_ENSTs,e_5mers,e_lines=set(),set(),set()
    for line in e_file:
        l=line.strip()
        e_ENSTs.add(l.split('_')[0])
        e_5mers.add(l.split('_')[2])
        e_lines.add(l)
    print('e_lines',len(e_lines))
    print('e_5mers',e_5mers)
    
    w_file=open(write_file+'.txt','w')
    s_lines=set()
    for to_select_sites_file in to_select_sites_files:
        s_file=open(to_select_sites_file+'.txt','r')
        for line in s_file:
            l=line.strip()
            if l.split('_')[0] not in e_ENSTs:
                continue
            if l.split('_')[2] not in e_5mers:
                continue
            if l not in e_lines:
                s_lines.add(l)

    print('s_lines',len(s_lines))
    s_lines=random.sample(list(s_lines),min(int(len(e_lines)/2),len(s_lines)))
    for line in s_lines:
        w_file.write(line+'\n')

In [17]:
get_negative_sites(['HepG2_ENST','A549_ENST'],'Hct116_ENST','others_reduced_by_Hct116_ENST')

e_lines 19326
e_5mers {'AAACT', 'GGACC', 'TGACC', 'AAACC', 'TGACT', 'GAACA', 'GGACT', 'AGACT', 'AAACA', 'GAACC', 'AGACC', 'TAACA', 'AGACA', 'TAACT', 'TAACC', 'TGACA', 'GGACA', 'GAACT'}
s_lines 71656
